# **🧬 Complete Cell Migration Analysis Pipeline**

# **👨‍💻 Author Details**

# **Subhajit Dutta, M.Sc.**

Institute of Cell Biology and Tumor Research (IBMZ),
University Medical Center Hamburg-Eppendorf (UKE)

University of Hamburg, Germany

Contact: dsubhajit.edu@gmail.com

# **📖 Introduction**

This notebook provides a comprehensive, interactive, and reproducible pipeline for analyzing cell migration datasets generated by TrackMate (an open-source tracking tool in Fiji/ImageJ).

TrackMate detects and tracks individual cells or particles across time-lapse microscopy images. It outputs CSV files containing positional coordinates (POSITION_X, POSITION_Y, POSITION_Z) and time/frame information (FRAME).
Our pipeline processes these CSV files to extract important biophysical and migratory parameters, visualize behavior over time, and statistically compare experimental conditions.

This tool is particularly useful for comparing different treatments, genetic modifications, or environmental conditions affecting cell motility.

**🛠️ Measurements and Features Calculated**

For each track (individual cell trajectory), the following features are extracted:

Mean Speed (μm/min): Average movement speed over time.

Total Displacement (μm): Straight-line distance from start to end.

Total Distance Travelled (μm): Cumulative path length.

Directionality (ratio): Straightness of the path (displacement / total distance).

Track Duration (frames): Time from first to last detection.

Asphericity: Shape irregularity of the trajectory (based on covariance of x and y).

Mean Square Displacement (MSD): Time-lagged spatial exploration behavior.

Mean Turning Angle (radians): Angular deviation between steps.

Velocity Vectors: Instantaneous movement vectors between frames.

Displacement Angle: Final angle between start and end points.

Additionally:

Instantaneous Speed Plot (per condition)

Directionality Polar Plot and Animated Rose Plots

Statistical comparisons between groups (auto-selected tests based on normality/homogeneity).

# **🧪 Usage Guidelines**

*Requirements*

Input data: TrackMate exported ***spot .csv files*** (one or multiple files per condition).

The notebook will automatically guide you interactively:

Upload your CSV files to Colab.

Specify the number of experimental conditions.

Assign files to each condition.

Run the full pipeline (either all tracks or equal number of tracks) to compute features, create plots, movies, and perform statistical analysis.

Outputs will be packaged into a downloadable .zip file containing:

CSV summary of extracted features

CSV of statistical test results

Box plots, strip plots, and rose plots (both raw and statistically annotated)

Animated MP4 movies of speed and directionality evolution

**📜 Usage Policy**

Free for academic use.

If you use this notebook or its output in your research:

Acknowledge its usage in your Methods section.

Cite the corresponding publication if a paper based on this pipeline appears (forthcoming).

Disclaimer:

The author is not responsible for any errors, misinterpretations, or modifications made to this pipeline after download.

Users are encouraged to validate outputs against their experimental expectations.

In [ ]:
######################## COMPLETE CELL MIGRATION ANALYSIS FOR ALL SPOTS ########################
print("Starting interactive cell migration analysis...")

# Install required packages
!pip install -q umap-learn seaborn>=0.12 scikit-posthocs ipywidgets
!apt-get install -qq ffmpeg > /dev/null

# Import libraries
import pandas as pd
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt
from matplotlib import animation
import seaborn as sns
import scipy.stats as stats
import scikit_posthocs as sp
from scipy.ndimage import gaussian_filter1d
import warnings
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output
warnings.filterwarnings('ignore')

# Configuration
data_dir = '/content'
output_dir = '/content/output'
os.makedirs(output_dir, exist_ok=True)

# ==================== INTERACTIVE SETUP ====================
print("\n=== Condition Setup ===")
print("1. Specify number of experimental conditions")
print("2. Name each condition")
print("3. Select CSV files for each condition from uploaded files\n")

# Get list of available CSV files
available_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]

# Widget for condition setup
condition_count = widgets.IntText(
    value=min(3, len(available_files)) if available_files else 1,
    description='Number of conditions:',
    disabled=False
)

condition_widgets = []
generate_btn = widgets.Button(description="Generate Input Fields")
run_btn = widgets.Button(description="Run Analysis", button_style='success')

def generate_fields(b):
    global condition_widgets
    clear_output()
    condition_widgets = []

    display(condition_count)

    for i in range(condition_count.value):
        name_widget = widgets.Text(
            description=f'Condition {i+1} Name:',
            style={'description_width': 'initial'}
        )

        # Create dropdown with available CSV files
        file_widget = widgets.SelectMultiple(
            options=available_files,
            description=f'Select CSV(s) for Condition {i+1}:',
            disabled=False,
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='50%')  # optional, makes it wider
        )


        condition_widgets.append((name_widget, file_widget))
        display(name_widget, file_widget)

    display(run_btn)

generate_btn.on_click(generate_fields)
display(condition_count, generate_btn)

# ==================== ANALYSIS FUNCTIONS ====================
def save_plot(fig, name, subfolder):
    if not hasattr(fig, 'savefig'):
        fig = plt.gcf()
    path = os.path.join(output_dir, subfolder, name)
    fig.savefig(path, bbox_inches='tight', dpi=300)
    plt.close(fig)

def compute_track_features(track_df):
    track_df = track_df.sort_values('FRAME')
    if len(track_df) < 2:
        return None

    # Clean and convert numeric columns
    for col in ['POSITION_X','POSITION_Y','POSITION_Z','FRAME']:
        if col in track_df.columns:
            track_df[col] = pd.to_numeric(track_df[col], errors='coerce')
            if col in ['POSITION_X','POSITION_Y','FRAME']:
                track_df = track_df.dropna(subset=[col])
    if len(track_df) < 2:
        return None

    # Position and time data
    x = track_df['POSITION_X'].values.astype(float)
    y = track_df['POSITION_Y'].values.astype(float)
    z = track_df.get('POSITION_Z', np.zeros(len(x))).astype(float)
    t = track_df['FRAME'].values.astype(float)

    # Calculate displacements and speeds
    delta_x = np.diff(x)
    delta_y = np.diff(y)
    delta_z = np.diff(z)
    delta_t = np.diff(t).astype(float)
    delta_t[delta_t == 0] = np.nan

    step_dist = np.sqrt(delta_x**2 + delta_y**2 + delta_z**2)
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        step_speed = step_dist / delta_t

    valid = ~np.isnan(step_speed)
    speeds = step_speed[valid]
    if len(speeds)==0:
        return None

    # Velocity and direction calculations
    velocity_vectors = np.column_stack((delta_x, delta_y)) / delta_t[:, None]
    step_angles = np.arctan2(velocity_vectors[:, 1], velocity_vectors[:, 0])
    turning_angles = np.abs(np.diff(step_angles))
    turning_angles = np.where(turning_angles > np.pi, 2*np.pi - turning_angles, turning_angles)

    # Core metrics
    mean_speed = np.nanmean(speeds)
    displacement = np.sqrt((x[-1]-x[0])**2 + (y[-1]-y[0])**2)
    total_distance = np.nansum(step_dist[valid])
    directionality = displacement / total_distance if total_distance > 0 else np.nan

    # MSD calculation
    time_lags = np.unique(t[1:][valid] - t[0])
    msd = np.zeros(len(time_lags))
    for i, lag in enumerate(time_lags):
        time_pairs = t[:, None] - t
        mask = (time_pairs == lag) & (np.triu(np.ones_like(time_pairs, dtype=bool), k=1))
        dx = x[:, None] - x
        dy = y[:, None] - y
        diffs = (dx**2 + dy**2)[mask]
        msd[i] = np.mean(diffs) if len(diffs) > 0 else np.nan

    # Shape metrics
    xy = np.column_stack((x, y))
    if len(xy) > 1:
        cov = np.cov(xy.T)
        eigvals = np.linalg.eigvals(cov)
        eigvals = sorted(eigvals, reverse=True)
        asphericity = (eigvals[0] - eigvals[1])**2 / sum(eigvals)**2 if sum(eigvals) > 0 else np.nan
    else:
        asphericity = np.nan

    # Final displacement angle
    dx = x[-1] - x[0]
    dy = y[-1] - y[0]
    displacement_angle = np.arctan2(dy, dx) if (dx != 0 or dy != 0) else np.nan

    return {
        'track_id': track_df['TRACK_ID'].iloc[0],
        'condition': track_df['condition'].iloc[0],
        'mean_speed': mean_speed,
        'displacement': displacement,
        'total_distance': total_distance,
        'directionality': directionality,
        'asphericity': asphericity,
        'duration': t[-1] - t[0],
        'mean_x': np.mean(x),
        'mean_y': np.mean(y),
        'step_speeds': speeds,
        'step_angles': step_angles,
        'turning_angles': turning_angles,
        'times': t[1:][valid],
        'velocity_vectors': velocity_vectors,
        'displacement_angle': displacement_angle,
        'mean_square_displacement': np.nanmean(msd),
        'mean_displacement': np.nanmean(step_dist[valid]),
        'track_straightness': directionality,
        'mean_turning_angle': np.nanmean(turning_angles) if len(turning_angles) > 0 else np.nan
    }

def create_instantaneous_speed_movie(features_df, color_palette, output_dir):
    print("Creating speed movie...")
    condition_data = {}
    max_speed, min_speed = 0, float('inf')

    for condition in color_palette:
        all_times, all_speeds = [], []
        for _, row in features_df[features_df['condition'] == condition].iterrows():
            all_times.extend(row['times'])
            all_speeds.extend(row['step_speeds'])

        df = pd.DataFrame({'time': all_times, 'speed': all_speeds})
        df = df.groupby('time')['speed'].mean().reset_index()
        df['smoothed_speed'] = gaussian_filter1d(df['speed'], sigma=1.5) if len(df) > 1 else df['speed']
        condition_data[condition] = df
        max_speed = max(max_speed, df['speed'].max())
        min_speed = min(min_speed, df['speed'].min())

    all_times = np.concatenate([df['time'] for df in condition_data.values()])
    time_points = np.linspace(min(all_times), max(all_times), 150)

    fig, ax = plt.subplots(figsize=(12, 8), dpi=300)
    plt.xlabel('Time (frames)')
    plt.ylabel('Speed (μm/min)')
    plt.title('Instantaneous Speed vs Time')

    raw_lines, smooth_lines, markers = {}, {}, {}
    for cond, color in color_palette.items():
        raw_lines[cond], = ax.plot([], [], lw=1.5, color=color, alpha=0.25)
        smooth_lines[cond], = ax.plot([], [], lw=2.5, color=color, alpha=0.9, label=cond)
        markers[cond], = ax.plot([], [], 'o', color=color, markersize=8)

    y_padding = (max_speed - min_speed) * 0.2
    ax.set_ylim(max(0, min_speed - y_padding), max_speed + y_padding)
    ax.set_xlim(min(all_times), max(all_times))
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    def animate(i):
        current_time = time_points[i]
        for cond, df in condition_data.items():
            mask = df['time'] <= current_time
            times = df['time'][mask]
            if len(times) > 0:
                raw_lines[cond].set_data(times, df['speed'][mask])
                smooth_lines[cond].set_data(times, df['smoothed_speed'][mask])
                if times.iloc[-1] >= current_time:
                    markers[cond].set_data([current_time], [df['smoothed_speed'][mask].iloc[-1]])
        return list(raw_lines.values()) + list(smooth_lines.values()) + list(markers.values())

    ani = animation.FuncAnimation(fig, animate, frames=len(time_points), interval=30, blit=True)
    output_path = os.path.join(output_dir, 'Without Stats', 'instantaneous_speed_movie.mp4')
    ani.save(output_path, writer='ffmpeg', dpi=300, bitrate=5000)
    plt.close()

def create_directionality_movie(features_df, palette, output_dir):
    print("Creating directionality movie...")
    def rose_plot(ax, angles, color, n_bins=24):
        hist, bins = np.histogram(angles, bins=n_bins, range=(-np.pi,np.pi))
        widths = np.diff(bins)
        ax.bar(bins[:-1], hist, width=widths, bottom=0, color=color, alpha=0.7, edgecolor='black', align='edge')

    cond_data = {}
    for cond in palette:
        times, angs = [], []
        for _, row in features_df[features_df['condition']==cond].iterrows():
            if not np.isnan(row['displacement_angle']):
                times.append(row['times'][-1])
                angs.append(row['displacement_angle'])
        cond_data[cond] = pd.DataFrame({'time':times,'angle':angs})

    all_t = np.unique(np.concatenate([d['time'] for d in cond_data.values()]))
    time_pts = np.linspace(all_t.min(), all_t.max(), 100)

    fig = plt.figure(figsize=(4*len(palette),4), dpi=300)
    axes = [fig.add_subplot(1,len(palette),i+1, projection='polar') for i in range(len(palette))]
    for ax in axes:
        ax.set_theta_zero_location('E')
        ax.set_theta_direction(-1)

    def animate(i):
        t = time_pts[i]
        for ax,(cond,color) in zip(axes, palette.items()):
            ax.clear()
            ax.set_theta_zero_location('E')
            ax.set_theta_direction(-1)
            ax.set_title(cond, pad=20)
            past = cond_data[cond][cond_data[cond]['time']<=t]
            if not past.empty:
                rose_plot(ax, past['angle'].values, color)
        return axes

    ani = animation.FuncAnimation(fig, animate, frames=len(time_pts), interval=100, blit=False)
    out = os.path.join(output_dir,'Without Stats','directionality_rose_movie.mp4')
    ani.save(out, writer='ffmpeg', dpi=300, bitrate=5000)
    plt.close(fig)

def run_statistical_tests(features_df, test_method='auto'):
    print("Running statistical tests...")
    statistical_results = []
    features_to_analyze = ['mean_speed', 'directionality', 'asphericity', 'duration',
                         'mean_square_displacement', 'mean_displacement',
                         'track_straightness', 'mean_turning_angle']
    conditions = features_df['condition'].unique()

    if len(conditions) < 2:
        return pd.DataFrame()

    for feature in features_to_analyze:
        groups = [features_df[features_df['condition'] == c][feature].dropna().values for c in conditions]
        valid_groups = [g for g in groups if len(g) > 0]
        if len(valid_groups) < 2:
            continue

        selected_test = test_method
        if test_method == 'auto':
            normal = all(len(g) > 3 and stats.shapiro(g)[1] > 0.05 for g in valid_groups)
            equal_var = stats.levene(*valid_groups).pvalue > 0.05 if normal else False

            if len(conditions) == 2:
                selected_test = 't-test' if (normal and equal_var) else 'welch' if normal else 'mannwhitney'
            else:
                selected_test = 'anova' if (normal and equal_var) else 'kruskal'

        try:
            if selected_test in ['t-test', 'welch']:
                result = stats.ttest_ind(*valid_groups[:2], equal_var=(selected_test == 't-test'))
                statistical_results.append({
                    'feature': feature,
                    'test_method': selected_test.upper(),
                    'condition1': conditions[0],
                    'condition2': conditions[1],
                    'p_value': result.pvalue,
                    'significance': '***' if result.pvalue < 0.001 else '**' if result.pvalue < 0.01 else '*' if result.pvalue < 0.05 else ''
                })
            elif selected_test == 'mannwhitney':
                result = stats.mannwhitneyu(*valid_groups[:2])
                statistical_results.append({
                    'feature': feature,
                    'test_method': selected_test.upper(),
                    'condition1': conditions[0],
                    'condition2': conditions[1],
                    'p_value': result.pvalue,
                    'significance': '***' if result.pvalue < 0.001 else '**' if result.pvalue < 0.01 else '*' if result.pvalue < 0.05 else ''
                })
            elif selected_test == 'anova':
                _, p_value = stats.f_oneway(*valid_groups)
                posthoc = sp.posthoc_tukey(features_df, val_col=feature, group_col='condition')
                for i in range(len(conditions)):
                    for j in range(i+1, len(conditions)):
                        cond1, cond2 = conditions[i], conditions[j]
                        p_val = posthoc.loc[cond1, cond2]
                        statistical_results.append({
                            'feature': feature,
                            'test_method': 'ANOVA-TUKEY',
                            'condition1': cond1,
                            'condition2': cond2,
                            'p_value': p_val,
                            'significance': '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else ''
                        })
            elif selected_test == 'kruskal':
                _, p_value = stats.kruskal(*valid_groups)
                posthoc = sp.posthoc_dunn(features_df, val_col=feature, group_col='condition', p_adjust='bonferroni')
                for i in range(len(conditions)):
                    for j in range(i+1, len(conditions)):
                        cond1, cond2 = conditions[i], conditions[j]
                        p_val = posthoc.loc[cond1, cond2]
                        statistical_results.append({
                            'feature': feature,
                            'test_method': 'KRUSKAL-DUNN',
                            'condition1': cond1,
                            'condition2': cond2,
                            'p_value': p_val,
                            'significance': '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else ''
                        })
        except Exception as e:
            print(f"Statistical test failed for {feature} ({selected_test}): {str(e)}")
            continue

    return pd.DataFrame(statistical_results)

def generate_plots(features_df, stats_df, color_palette):
    print("Generating plots...")
    plt.rcParams.update({
        'font.size': 12,
        'axes.titlesize': 14,
        'axes.labelsize': 12,
        'xtick.labelsize': 10,
        'ytick.labelsize': 10,
        'figure.dpi': 300,
        'savefig.dpi': 300,
        'figure.constrained_layout.use': True
    })

    features = ['mean_speed', 'directionality', 'asphericity', 'duration',
               'mean_square_displacement', 'mean_displacement',
               'track_straightness', 'mean_turning_angle']

    # Plots without stats
    for feat in features:
        fig, ax = plt.subplots(figsize=(8, 7))
        sns.boxplot(x='condition', y=feat, data=features_df, palette=color_palette, ax=ax)
        sns.stripplot(x='condition', y=feat, data=features_df, palette=color_palette,
                     edgecolor='black', linewidth=0.5, size=4, alpha=0.7, ax=ax)
        ax.set_xlabel('Condition')
        ax.set_ylabel(' '.join(word.capitalize() for word in feat.split('_')))
        ax.set_title(' '.join(word.capitalize() for word in feat.split('_')), fontweight='bold')
        plt.tight_layout()
        save_plot(fig, f'{feat}_without_stats.png', 'Without Stats')

    # Plots with stats
    if not stats_df.empty and 'significance' in stats_df.columns:
        for feat in features:
            fig, ax = plt.subplots(figsize=(10, 9))
            sns.boxplot(x='condition', y=feat, data=features_df, palette=color_palette, ax=ax)
            sns.stripplot(x='condition', y=feat, data=features_df, palette=color_palette,
                         edgecolor='black', linewidth=0.5, size=4, alpha=0.7, ax=ax)

            ax.set_xlabel('Condition')
            ax.set_ylabel(' '.join(word.capitalize() for word in feat.split('_')))

            ymin, ymax = ax.get_ylim()
            y_pad = (ymax - ymin) * 0.4
            ax.set_ylim(ymin, ymax + y_pad)

            pairs = [(c1, c2) for i, c1 in enumerate(color_palette) for j, c2 in enumerate(color_palette) if i < j]
            y = ymax + (ymax - ymin) * 0.1
            for c1, c2 in pairs:
                row = stats_df.query("feature == @feat and condition1 == @c1 and condition2 == @c2")
                if not row.empty and row['significance'].iloc[0]:
                    x1 = list(color_palette).index(c1)
                    x2 = list(color_palette).index(c2)
                    stars = row['significance'].iloc[0]
                    ax.plot([x1, x1, x2, x2], [y, y + 0.05*(ymax - ymin), y + 0.05*(ymax - ymin), y], 'k-')
                    ax.text((x1 + x2)/2, y + 0.06*(ymax - ymin), stars, ha='center', va='bottom')
                    y += 0.08 * (ymax - ymin)

            ax.set_title(f"{' '.join(word.capitalize() for word in feat.split('_'))} (With Stats)", fontweight='bold')
            plt.tight_layout()
            save_plot(fig, f'{feat}_with_stats.png', 'With Stats')

    # Instantaneous speed plot
    fig, ax = plt.subplots(figsize=(12,8), dpi=300)
    for cond, color in color_palette.items():
        subset = features_df[features_df['condition']==cond]
        times = np.concatenate(subset['times'].tolist())
        speeds = np.concatenate(subset['step_speeds'].tolist())
        df = pd.DataFrame({'time':times,'speed':speeds}).groupby('time')['speed'].mean().reset_index()
        if len(df)>1:
            sm = gaussian_filter1d(df['speed'], sigma=1.5)
            ax.plot(df['time'], sm, linewidth=2.5, color=color, alpha=1.0, label=cond)
    ymin, ymax = ax.get_ylim()
    pad = (ymax-ymin)*0.2
    ax.set_ylim(max(0,ymin-pad), ymax+pad)
    ax.set_xlabel('Time (frames)')
    ax.set_ylabel('Speed (μm/min)')
    ax.set_title('Instantaneous Speed vs Time')
    ax.legend(list(color_palette.keys()), bbox_to_anchor=(1.05,1), loc='upper left')
    save_plot(fig, 'instantaneous_speed_vs_time.png','Without Stats')

    # Directionality polar plot
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(111, projection='polar')
    ax.set_theta_zero_location('E')
    ax.set_theta_direction(-1)
    ax.set_title('Cell Migration Directionality', pad=20)
    for cond, color in color_palette.items():
        angles = [r for r in features_df[features_df['condition']==cond]['displacement_angle'] if not np.isnan(r)]
        if angles:
            hist, bins = np.histogram(angles, bins=24, range=(-np.pi, np.pi))
            centers = (bins[:-1]+bins[1:])/2
            hist = hist / hist.sum() if hist.sum()>0 else hist
            data_angles = np.concatenate([centers, [centers[0]]])
            data_hist = np.concatenate([hist, [hist[0]]])
            ax.plot(data_angles, data_hist, 'o-', label=cond)
    ax.legend(bbox_to_anchor=(1.15,1), loc='upper left')
    save_plot(fig, 'directionality_polar_plot.png', 'Without Stats')

    # Filled-rose plot
    def rose_plot(ax, angles, color, n_bins=24):
        hist, bins = np.histogram(angles, bins=n_bins, range=(-np.pi,np.pi))
        widths = np.diff(bins)
        ax.bar(bins[:-1], hist, width=widths, bottom=0, color=color, alpha=0.7, edgecolor='black', align='edge')

    fig = plt.figure(figsize=(4*len(color_palette),4), dpi=300)
    for i,(cond,color) in enumerate(color_palette.items(),1):
        ax = fig.add_subplot(1,len(color_palette),i, projection='polar')
        ax.set_theta_zero_location('E')
        ax.set_theta_direction(-1)
        angles = features_df.query("condition==@cond")['displacement_angle'].dropna().values
        rose_plot(ax, angles, color)
        ax.set_title(cond, pad=20)
    save_plot(fig, 'directionality_rose_plot.png','Without Stats')

# ==================== MAIN EXECUTION ====================
def run_analysis(b):
    clear_output()
    print("Setting up analysis...")

    # Reset output folder
    shutil.rmtree(output_dir, ignore_errors=True)
    os.makedirs(os.path.join(output_dir, "Without Stats"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "With Stats"), exist_ok=True)

    # Process selected files
    CONDITION_GROUPS = {}
    for name_widget, file_widget in condition_widgets:
        condition_name = name_widget.value.strip()
        selected_files = list(file_widget.value)

        if not condition_name:
            print(f"Warning: Empty condition name skipped")
            continue

        if selected_files:
            CONDITION_GROUPS[condition_name] = selected_files
        else:
            print(f"Warning: No file selected for {condition_name}")

    if not CONDITION_GROUPS:
        print("Error: No valid conditions with files provided")
        return

    print("\nConditions to analyze:")
    for cond, files_list in CONDITION_GROUPS.items():
        print(f"- {cond}: {files_list}")

    # Run analysis pipeline
    STAT_METHOD = 'auto'

    # Verify inputs
    all_files = [f for fl in CONDITION_GROUPS.values() for f in fl]
    missing = [f for f in all_files if not os.path.exists(os.path.join(data_dir, f))]
    if missing:
        raise FileNotFoundError(f"Missing: {missing}")

    # Compute features
    all_feats = []
    for cond, files_list in CONDITION_GROUPS.items():
        for f in files_list:
            df = pd.read_csv(os.path.join(data_dir, f), low_memory=False)
            df.columns = df.columns.str.strip()
            df['condition'] = cond
            df['TRACK_ID'] = pd.to_numeric(df['TRACK_ID'], errors='coerce').dropna().astype(int)
            for tid in df['TRACK_ID'].unique():
                feats = compute_track_features(df[df['TRACK_ID'] == tid])
                if feats:
                    all_feats.append(feats)

    features_df = pd.DataFrame(all_feats)
    features_df.to_csv(os.path.join(output_dir, 'track_features.csv'), index=False)

    # Only run stats if >1 condition
    stats_df = run_statistical_tests(features_df, STAT_METHOD) if len(CONDITION_GROUPS) > 1 else pd.DataFrame()
    stats_df.to_csv(os.path.join(output_dir, 'statistical_results.csv'), index=False)

    # Generate results
    conds = list(CONDITION_GROUPS.keys())
    cmap = plt.cm.tab20(np.linspace(0, 1, len(conds)))
    palette = {conds[i]: cmap[i] for i in range(len(conds))}

    generate_plots(features_df, stats_df, palette)
    create_instantaneous_speed_movie(features_df, palette, output_dir)
    create_directionality_movie(features_df, palette, output_dir)

    # Provide download link
    print("\nAnalysis complete! Download results:")
    !zip -r /content/results.zip /content/output
    files.download('/content/results.zip')


run_btn.on_click(run_analysis)

In [ ]:
######################## COMPLETE CELL MIGRATION ANALYSIS FOR EQUAL NUMBER OF SPOTS ########################
print("Starting interactive cell migration analysis...")

# Install required packages
!pip install -q umap-learn seaborn>=0.12 scikit-posthocs ipywidgets
!apt-get install -qq ffmpeg > /dev/null

# Import libraries
import pandas as pd
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt
from matplotlib import animation
import seaborn as sns
import scipy.stats as stats
import scikit_posthocs as sp
from scipy.ndimage import gaussian_filter1d
import warnings
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output
warnings.filterwarnings('ignore')

# Configuration
data_dir = '/content'
output_dir = '/content/output'
os.makedirs(output_dir, exist_ok=True)

# ==================== INTERACTIVE SETUP ====================
print("\n=== Condition Setup ===")
print("1. Specify number of experimental conditions")
print("2. Name each condition")
print("3. Select CSV files for each condition from uploaded files\n")

# Get list of available CSV files
available_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]

# Widget for condition setup
condition_count = widgets.IntText(
    value=min(3, len(available_files)) if available_files else 1,
    description='Number of conditions:',
    disabled=False
)

condition_widgets = []
generate_btn = widgets.Button(description="Generate Input Fields")
run_btn = widgets.Button(description="Run Analysis", button_style='success')

def generate_fields(b):
    global condition_widgets
    clear_output()
    condition_widgets = []

    display(condition_count)

    for i in range(condition_count.value):
        name_widget = widgets.Text(
            description=f'Condition {i+1} Name:',
            style={'description_width': 'initial'}
        )

        # Create dropdown with available CSV files
        file_widget = widgets.SelectMultiple(
            options=available_files,
            description=f'Select CSV(s) for Condition {i+1}:',
            disabled=False,
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='50%')  # optional, makes it wider
        )


        condition_widgets.append((name_widget, file_widget))
        display(name_widget, file_widget)

    display(run_btn)

generate_btn.on_click(generate_fields)
display(condition_count, generate_btn)

# ==================== ANALYSIS FUNCTIONS ====================
def save_plot(fig, name, subfolder):
    if not hasattr(fig, 'savefig'):
        fig = plt.gcf()
    path = os.path.join(output_dir, subfolder, name)
    fig.savefig(path, bbox_inches='tight', dpi=300)
    plt.close(fig)

def compute_track_features(track_df):
    track_df = track_df.sort_values('FRAME')
    if len(track_df) < 2:
        return None

    # Clean and convert numeric columns
    for col in ['POSITION_X','POSITION_Y','POSITION_Z','FRAME']:
        if col in track_df.columns:
            track_df[col] = pd.to_numeric(track_df[col], errors='coerce')
            if col in ['POSITION_X','POSITION_Y','FRAME']:
                track_df = track_df.dropna(subset=[col])
    if len(track_df) < 2:
        return None

    # Position and time data
    x = track_df['POSITION_X'].values.astype(float)
    y = track_df['POSITION_Y'].values.astype(float)
    z = track_df.get('POSITION_Z', np.zeros(len(x))).astype(float)
    t = track_df['FRAME'].values.astype(float)

    # Calculate displacements and speeds
    delta_x = np.diff(x)
    delta_y = np.diff(y)
    delta_z = np.diff(z)
    delta_t = np.diff(t).astype(float)
    delta_t[delta_t == 0] = np.nan

    step_dist = np.sqrt(delta_x**2 + delta_y**2 + delta_z**2)
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        step_speed = step_dist / delta_t

    valid = ~np.isnan(step_speed)
    speeds = step_speed[valid]
    if len(speeds)==0:
        return None

    # Velocity and direction calculations
    velocity_vectors = np.column_stack((delta_x, delta_y)) / delta_t[:, None]
    step_angles = np.arctan2(velocity_vectors[:, 1], velocity_vectors[:, 0])
    turning_angles = np.abs(np.diff(step_angles))
    turning_angles = np.where(turning_angles > np.pi, 2*np.pi - turning_angles, turning_angles)

    # Core metrics
    mean_speed = np.nanmean(speeds)
    displacement = np.sqrt((x[-1]-x[0])**2 + (y[-1]-y[0])**2)
    total_distance = np.nansum(step_dist[valid])
    directionality = displacement / total_distance if total_distance > 0 else np.nan

    # MSD calculation
    time_lags = np.unique(t[1:][valid] - t[0])
    msd = np.zeros(len(time_lags))
    for i, lag in enumerate(time_lags):
        time_pairs = t[:, None] - t
        mask = (time_pairs == lag) & (np.triu(np.ones_like(time_pairs, dtype=bool), k=1))
        dx = x[:, None] - x
        dy = y[:, None] - y
        diffs = (dx**2 + dy**2)[mask]
        msd[i] = np.mean(diffs) if len(diffs) > 0 else np.nan

    # Shape metrics
    xy = np.column_stack((x, y))
    if len(xy) > 1:
        cov = np.cov(xy.T)
        eigvals = np.linalg.eigvals(cov)
        eigvals = sorted(eigvals, reverse=True)
        asphericity = (eigvals[0] - eigvals[1])**2 / sum(eigvals)**2 if sum(eigvals) > 0 else np.nan
    else:
        asphericity = np.nan

    # Final displacement angle
    dx = x[-1] - x[0]
    dy = y[-1] - y[0]
    displacement_angle = np.arctan2(dy, dx) if (dx != 0 or dy != 0) else np.nan

    return {
        'track_id': track_df['TRACK_ID'].iloc[0],
        'condition': track_df['condition'].iloc[0],
        'mean_speed': mean_speed,
        'displacement': displacement,
        'total_distance': total_distance,
        'directionality': directionality,
        'asphericity': asphericity,
        'duration': t[-1] - t[0],
        'mean_x': np.mean(x),
        'mean_y': np.mean(y),
        'step_speeds': speeds,
        'step_angles': step_angles,
        'turning_angles': turning_angles,
        'times': t[1:][valid],
        'velocity_vectors': velocity_vectors,
        'displacement_angle': displacement_angle,
        'mean_square_displacement': np.nanmean(msd),
        'mean_displacement': np.nanmean(step_dist[valid]),
        'track_straightness': directionality,
        'mean_turning_angle': np.nanmean(turning_angles) if len(turning_angles) > 0 else np.nan
    }

def create_instantaneous_speed_movie(features_df, color_palette, output_dir):
    print("Creating speed movie...")
    condition_data = {}
    max_speed, min_speed = 0, float('inf')

    for condition in color_palette:
        all_times, all_speeds = [], []
        for _, row in features_df[features_df['condition'] == condition].iterrows():
            all_times.extend(row['times'])
            all_speeds.extend(row['step_speeds'])

        df = pd.DataFrame({'time': all_times, 'speed': all_speeds})
        df = df.groupby('time')['speed'].mean().reset_index()
        df['smoothed_speed'] = gaussian_filter1d(df['speed'], sigma=1.5) if len(df) > 1 else df['speed']
        condition_data[condition] = df
        max_speed = max(max_speed, df['speed'].max())
        min_speed = min(min_speed, df['speed'].min())

    all_times = np.concatenate([df['time'] for df in condition_data.values()])
    time_points = np.linspace(min(all_times), max(all_times), 150)

    fig, ax = plt.subplots(figsize=(12, 8), dpi=300)
    plt.xlabel('Time (frames)')
    plt.ylabel('Speed (μm/min)')
    plt.title('Instantaneous Speed vs Time')

    raw_lines, smooth_lines, markers = {}, {}, {}
    for cond, color in color_palette.items():
        raw_lines[cond], = ax.plot([], [], lw=1.5, color=color, alpha=0.25)
        smooth_lines[cond], = ax.plot([], [], lw=2.5, color=color, alpha=0.9, label=cond)
        markers[cond], = ax.plot([], [], 'o', color=color, markersize=8)

    y_padding = (max_speed - min_speed) * 0.2
    ax.set_ylim(max(0, min_speed - y_padding), max_speed + y_padding)
    ax.set_xlim(min(all_times), max(all_times))
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    def animate(i):
        current_time = time_points[i]
        for cond, df in condition_data.items():
            mask = df['time'] <= current_time
            times = df['time'][mask]
            if len(times) > 0:
                raw_lines[cond].set_data(times, df['speed'][mask])
                smooth_lines[cond].set_data(times, df['smoothed_speed'][mask])
                if times.iloc[-1] >= current_time:
                    markers[cond].set_data([current_time], [df['smoothed_speed'][mask].iloc[-1]])
        return list(raw_lines.values()) + list(smooth_lines.values()) + list(markers.values())

    ani = animation.FuncAnimation(fig, animate, frames=len(time_points), interval=30, blit=True)
    output_path = os.path.join(output_dir, 'Without Stats', 'instantaneous_speed_movie.mp4')
    ani.save(output_path, writer='ffmpeg', dpi=300, bitrate=5000)
    plt.close()

def create_directionality_movie(features_df, palette, output_dir):
    print("Creating directionality movie...")
    def rose_plot(ax, angles, color, n_bins=24):
        hist, bins = np.histogram(angles, bins=n_bins, range=(-np.pi,np.pi))
        widths = np.diff(bins)
        ax.bar(bins[:-1], hist, width=widths, bottom=0, color=color, alpha=0.7, edgecolor='black', align='edge')

    cond_data = {}
    for cond in palette:
        times, angs = [], []
        for _, row in features_df[features_df['condition']==cond].iterrows():
            if not np.isnan(row['displacement_angle']):
                times.append(row['times'][-1])
                angs.append(row['displacement_angle'])
        cond_data[cond] = pd.DataFrame({'time':times,'angle':angs})

    all_t = np.unique(np.concatenate([d['time'] for d in cond_data.values()]))
    time_pts = np.linspace(all_t.min(), all_t.max(), 100)

    fig = plt.figure(figsize=(4*len(palette),4), dpi=300)
    axes = [fig.add_subplot(1,len(palette),i+1, projection='polar') for i in range(len(palette))]
    for ax in axes:
        ax.set_theta_zero_location('E')
        ax.set_theta_direction(-1)

    def animate(i):
        t = time_pts[i]
        for ax,(cond,color) in zip(axes, palette.items()):
            ax.clear()
            ax.set_theta_zero_location('E')
            ax.set_theta_direction(-1)
            ax.set_title(cond, pad=20)
            past = cond_data[cond][cond_data[cond]['time']<=t]
            if not past.empty:
                rose_plot(ax, past['angle'].values, color)
        return axes

    ani = animation.FuncAnimation(fig, animate, frames=len(time_pts), interval=100, blit=False)
    out = os.path.join(output_dir,'Without Stats','directionality_rose_movie.mp4')
    ani.save(out, writer='ffmpeg', dpi=300, bitrate=5000)
    plt.close(fig)

def run_statistical_tests(features_df, test_method='auto'):
    print("Running statistical tests...")
    statistical_results = []
    features_to_analyze = ['mean_speed', 'directionality', 'asphericity', 'duration',
                         'mean_square_displacement', 'mean_displacement',
                         'track_straightness', 'mean_turning_angle']
    conditions = features_df['condition'].unique()

    if len(conditions) < 2:
        return pd.DataFrame()

    for feature in features_to_analyze:
        groups = [features_df[features_df['condition'] == c][feature].dropna().values for c in conditions]
        valid_groups = [g for g in groups if len(g) > 0]
        if len(valid_groups) < 2:
            continue

        selected_test = test_method
        if test_method == 'auto':
            normal = all(len(g) > 3 and stats.shapiro(g)[1] > 0.05 for g in valid_groups)
            equal_var = stats.levene(*valid_groups).pvalue > 0.05 if normal else False

            if len(conditions) == 2:
                selected_test = 't-test' if (normal and equal_var) else 'welch' if normal else 'mannwhitney'
            else:
                selected_test = 'anova' if (normal and equal_var) else 'kruskal'

        try:
            if selected_test in ['t-test', 'welch']:
                result = stats.ttest_ind(*valid_groups[:2], equal_var=(selected_test == 't-test'))
                statistical_results.append({
                    'feature': feature,
                    'test_method': selected_test.upper(),
                    'condition1': conditions[0],
                    'condition2': conditions[1],
                    'p_value': result.pvalue,
                    'significance': '***' if result.pvalue < 0.001 else '**' if result.pvalue < 0.01 else '*' if result.pvalue < 0.05 else ''
                })
            elif selected_test == 'mannwhitney':
                result = stats.mannwhitneyu(*valid_groups[:2])
                statistical_results.append({
                    'feature': feature,
                    'test_method': selected_test.upper(),
                    'condition1': conditions[0],
                    'condition2': conditions[1],
                    'p_value': result.pvalue,
                    'significance': '***' if result.pvalue < 0.001 else '**' if result.pvalue < 0.01 else '*' if result.pvalue < 0.05 else ''
                })
            elif selected_test == 'anova':
                _, p_value = stats.f_oneway(*valid_groups)
                posthoc = sp.posthoc_tukey(features_df, val_col=feature, group_col='condition')
                for i in range(len(conditions)):
                    for j in range(i+1, len(conditions)):
                        cond1, cond2 = conditions[i], conditions[j]
                        p_val = posthoc.loc[cond1, cond2]
                        statistical_results.append({
                            'feature': feature,
                            'test_method': 'ANOVA-TUKEY',
                            'condition1': cond1,
                            'condition2': cond2,
                            'p_value': p_val,
                            'significance': '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else ''
                        })
            elif selected_test == 'kruskal':
                _, p_value = stats.kruskal(*valid_groups)
                posthoc = sp.posthoc_dunn(features_df, val_col=feature, group_col='condition', p_adjust='bonferroni')
                for i in range(len(conditions)):
                    for j in range(i+1, len(conditions)):
                        cond1, cond2 = conditions[i], conditions[j]
                        p_val = posthoc.loc[cond1, cond2]
                        statistical_results.append({
                            'feature': feature,
                            'test_method': 'KRUSKAL-DUNN',
                            'condition1': cond1,
                            'condition2': cond2,
                            'p_value': p_val,
                            'significance': '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else ''
                        })
        except Exception as e:
            print(f"Statistical test failed for {feature} ({selected_test}): {str(e)}")
            continue

    return pd.DataFrame(statistical_results)

def generate_plots(features_df, stats_df, color_palette):
    print("Generating plots...")
    plt.rcParams.update({
        'font.size': 12,
        'axes.titlesize': 14,
        'axes.labelsize': 12,
        'xtick.labelsize': 10,
        'ytick.labelsize': 10,
        'figure.dpi': 300,
        'savefig.dpi': 300,
        'figure.constrained_layout.use': True
    })

    features = ['mean_speed', 'directionality', 'asphericity', 'duration',
               'mean_square_displacement', 'mean_displacement',
               'track_straightness', 'mean_turning_angle']

    # Plots without stats
    for feat in features:
        fig, ax = plt.subplots(figsize=(8, 7))
        sns.boxplot(x='condition', y=feat, data=features_df, palette=color_palette, ax=ax)
        sns.stripplot(x='condition', y=feat, data=features_df, palette=color_palette,
                     edgecolor='black', linewidth=0.5, size=4, alpha=0.7, ax=ax)
        ax.set_xlabel('Condition')
        ax.set_ylabel(' '.join(word.capitalize() for word in feat.split('_')))
        ax.set_title(' '.join(word.capitalize() for word in feat.split('_')), fontweight='bold')
        plt.tight_layout()
        save_plot(fig, f'{feat}_without_stats.png', 'Without Stats')

    # Plots with stats
    if not stats_df.empty and 'significance' in stats_df.columns:
        for feat in features:
            fig, ax = plt.subplots(figsize=(10, 9))
            sns.boxplot(x='condition', y=feat, data=features_df, palette=color_palette, ax=ax)
            sns.stripplot(x='condition', y=feat, data=features_df, palette=color_palette,
                         edgecolor='black', linewidth=0.5, size=4, alpha=0.7, ax=ax)

            ax.set_xlabel('Condition')
            ax.set_ylabel(' '.join(word.capitalize() for word in feat.split('_')))

            ymin, ymax = ax.get_ylim()
            y_pad = (ymax - ymin) * 0.4
            ax.set_ylim(ymin, ymax + y_pad)

            pairs = [(c1, c2) for i, c1 in enumerate(color_palette) for j, c2 in enumerate(color_palette) if i < j]
            y = ymax + (ymax - ymin) * 0.1
            for c1, c2 in pairs:
                row = stats_df.query("feature == @feat and condition1 == @c1 and condition2 == @c2")
                if not row.empty and row['significance'].iloc[0]:
                    x1 = list(color_palette).index(c1)
                    x2 = list(color_palette).index(c2)
                    stars = row['significance'].iloc[0]
                    ax.plot([x1, x1, x2, x2], [y, y + 0.05*(ymax - ymin), y + 0.05*(ymax - ymin), y], 'k-')
                    ax.text((x1 + x2)/2, y + 0.06*(ymax - ymin), stars, ha='center', va='bottom')
                    y += 0.08 * (ymax - ymin)

            ax.set_title(f"{' '.join(word.capitalize() for word in feat.split('_'))} (With Stats)", fontweight='bold')
            plt.tight_layout()
            save_plot(fig, f'{feat}_with_stats.png', 'With Stats')

    # Instantaneous speed plot
    fig, ax = plt.subplots(figsize=(12,8), dpi=300)
    for cond, color in color_palette.items():
        subset = features_df[features_df['condition']==cond]
        times = np.concatenate(subset['times'].tolist())
        speeds = np.concatenate(subset['step_speeds'].tolist())
        df = pd.DataFrame({'time':times,'speed':speeds}).groupby('time')['speed'].mean().reset_index()
        if len(df)>1:
            sm = gaussian_filter1d(df['speed'], sigma=1.5)
            ax.plot(df['time'], sm, linewidth=2.5, color=color, alpha=1.0, label=cond)
    ymin, ymax = ax.get_ylim()
    pad = (ymax-ymin)*0.2
    ax.set_ylim(max(0,ymin-pad), ymax+pad)
    ax.set_xlabel('Time (frames)')
    ax.set_ylabel('Speed (μm/min)')
    ax.set_title('Instantaneous Speed vs Time')
    ax.legend(list(color_palette.keys()), bbox_to_anchor=(1.05,1), loc='upper left')
    save_plot(fig, 'instantaneous_speed_vs_time.png','Without Stats')

    # Directionality polar plot
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(111, projection='polar')
    ax.set_theta_zero_location('E')
    ax.set_theta_direction(-1)
    ax.set_title('Cell Migration Directionality', pad=20)
    for cond, color in color_palette.items():
        angles = [r for r in features_df[features_df['condition']==cond]['displacement_angle'] if not np.isnan(r)]
        if angles:
            hist, bins = np.histogram(angles, bins=24, range=(-np.pi, np.pi))
            centers = (bins[:-1]+bins[1:])/2
            hist = hist / hist.sum() if hist.sum()>0 else hist
            data_angles = np.concatenate([centers, [centers[0]]])
            data_hist = np.concatenate([hist, [hist[0]]])
            ax.plot(data_angles, data_hist, 'o-', label=cond)
    ax.legend(bbox_to_anchor=(1.15,1), loc='upper left')
    save_plot(fig, 'directionality_polar_plot.png', 'Without Stats')

    # Filled-rose plot
    def rose_plot(ax, angles, color, n_bins=24):
        hist, bins = np.histogram(angles, bins=n_bins, range=(-np.pi,np.pi))
        widths = np.diff(bins)
        ax.bar(bins[:-1], hist, width=widths, bottom=0, color=color, alpha=0.7, edgecolor='black', align='edge')

    fig = plt.figure(figsize=(4*len(color_palette),4), dpi=300)
    for i,(cond,color) in enumerate(color_palette.items(),1):
        ax = fig.add_subplot(1,len(color_palette),i, projection='polar')
        ax.set_theta_zero_location('E')
        ax.set_theta_direction(-1)
        angles = features_df.query("condition==@cond")['displacement_angle'].dropna().values
        rose_plot(ax, angles, color)
        ax.set_title(cond, pad=20)
    save_plot(fig, 'directionality_rose_plot.png','Without Stats')

# ==================== MAIN EXECUTION ====================
def run_analysis(b):
    clear_output()
    print("Setting up analysis...")

    # Reset output folder
    shutil.rmtree(output_dir, ignore_errors=True)
    os.makedirs(os.path.join(output_dir, "Without Stats"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "With Stats"), exist_ok=True)

    # Process selected files
    CONDITION_GROUPS = {}
    for name_widget, file_widget in condition_widgets:
        condition_name = name_widget.value.strip()
        selected_files = list(file_widget.value)

        if not condition_name:
            print(f"Warning: Empty condition name skipped")
            continue

        if selected_files:
            CONDITION_GROUPS[condition_name] = selected_files
        else:
            print(f"Warning: No file selected for {condition_name}")

    if not CONDITION_GROUPS:
        print("Error: No valid conditions with files provided")
        return

    print("\nConditions to analyze:")
    for cond, files_list in CONDITION_GROUPS.items():
        print(f"- {cond}: {files_list}")

    # Run analysis pipeline
    STAT_METHOD = 'auto'

    # Verify inputs
    all_files = [f for fl in CONDITION_GROUPS.values() for f in fl]
    missing = [f for f in all_files if not os.path.exists(os.path.join(data_dir, f))]
    if missing:
        raise FileNotFoundError(f"Missing: {missing}")

    # Compute features
    all_feats = []
    for cond, files_list in CONDITION_GROUPS.items():
        for f in files_list:
            df = pd.read_csv(os.path.join(data_dir, f), low_memory=False)
            df.columns = df.columns.str.strip()
            df['condition'] = cond
            df['TRACK_ID'] = pd.to_numeric(df['TRACK_ID'], errors='coerce').dropna().astype(int)
            for tid in df['TRACK_ID'].unique():
                feats = compute_track_features(df[df['TRACK_ID'] == tid])
                if feats:
                    all_feats.append(feats)

    features_df = pd.DataFrame(all_feats)
    # randomly down-sample so each condition has the same number of tracks
    min_count = features_df['condition'].value_counts().min()
    features_df = (
        features_df
        .groupby('condition', group_keys=False)
        .apply(lambda grp: grp.sample(n=min_count, random_state=42))
        .reset_index(drop=True)
    )
    features_df.to_csv(os.path.join(output_dir, 'track_features.csv'), index=False)

    # Only run stats if >1 condition
    stats_df = run_statistical_tests(features_df, STAT_METHOD) if len(CONDITION_GROUPS) > 1 else pd.DataFrame()
    stats_df.to_csv(os.path.join(output_dir, 'statistical_results.csv'), index=False)

    # Generate results
    conds = list(CONDITION_GROUPS.keys())
    cmap = plt.cm.tab20(np.linspace(0, 1, len(conds)))
    palette = {conds[i]: cmap[i] for i in range(len(conds))}

    generate_plots(features_df, stats_df, palette)
    create_instantaneous_speed_movie(features_df, palette, output_dir)
    create_directionality_movie(features_df, palette, output_dir)

    # Provide download link
    print("\nAnalysis complete! Download results:")
    !zip -r /content/results.zip /content/output
    files.download('/content/results.zip')


run_btn.on_click(run_analysis)